# Getting Started With `riweather`

In [1]:
import matplotlib.pyplot as plt

import riweather

%matplotlib inline

Suppose you need weather data at a certain site. To list all weather stations in order of their distance to that site, use [`riweather.rank_stations()`][riweather.rank_stations].

In [2]:
my_lat, my_lon = 39.98, -105.13
ranked_stations = riweather.rank_stations(my_lat, my_lon)

In [3]:
print("Number of stations returned:", ranked_stations.shape[0])
ranked_stations.head()

This gives us the name and USAF identifiers of all ISD weather stations in the United States, along with their latitudes and longitudes and their distances (in meters) to the target site. The "years" column is a list of years for which there is available weather data from that station. The "quality" column contains the quality of the data file for the corresponding year---this is based on the number of observations in the file.

You probably don't want all three thousand weather stations that are returned, so let's be more efficient and limit our search to stations that have data for 2022 and stations that are within 20 km.

In [4]:
ranked_stations = riweather.rank_stations(
    my_lat,
    my_lon,
    year=2022,
    max_distance_m=20000,
)
ranked_stations

Select the closest station, Erie Municipal Airport, using [`riweather.select_station()`][riweather.select_station].

In [5]:
station = riweather.select_station(ranked_stations, rank=0)
station

Let's see how complete we can expect the data to be when we pull it by looking at the quality report.

In [6]:
station.quality_report()

The data for 2022 is `"high"` quality, with more than 2,000 observations in each month. Assuming they are evenly spaced, that corresponds to one observation roughly every 20 minutes.

Now we can use the [`Station.fetch_temp_data()`][riweather.Station.fetch_temp_data] method to get temperature data in degrees Fahrenheit.

In [7]:
ts = station.fetch_temp_data(2022, value="temperature", scale="F")
ts.head()

It's important to note that **the timestamps are always in UTC**. The temperature data is hourly by default, but `riweather` can resample it to a different frequency as well.

In [8]:
ts_30min = station.fetch_temp_data(
    2022,
    value="temperature",
    scale="F",
    period="30min",
)
ts_30min.head()

In [9]:
ts_30min.plot(
    title=f"Air temperature at {station.name.title()}, 2022",
    ylabel="Air temperature (degrees F)",
)
plt.show()

We can verify the location of our site and the nearest weather stations by plotting them on a map with [`riweather.plot_stations()`][riweather.plot_stations].

In [10]:
riweather.plot_stations(my_lat, my_lon, ranked_stations, distance_unit="mi")

Only have a ZIP code for your site and not a precise latitude and longitude? No problem. You can use [`riweather.zcta_to_lat_lon()`][riweather.zcta_to_lat_lon] to get an approximate location.

In [11]:
my_lat, my_lon = riweather.zcta_to_lat_lon("80027")
print(f"{my_lat:.3f}", f"{my_lon:.3f}")

In [12]:
ranked_stations = riweather.rank_stations(my_lat, my_lon, max_distance_m=20000)
riweather.plot_stations(my_lat, my_lon, ranked_stations, distance_unit="mi")